In [2]:
from bs4 import BeautifulSoup
import requests
from requests import get
import urllib.parse
from urllib.error import HTTPError
import json
import time
import pandas as pd
import numpy as np

Scraping bestplaces.net's political leanings which is based on recent voting in 
national elections, federal campaign contributions by local residents, and 
consumer personality profiles.

In [3]:
top_cities = pd.read_csv('Top5000Population.csv')
states = pd.read_csv('data.csv')
states['Cities'] = np.empty((len(states), 0)).tolist()
top_cities['State']=np.empty((len(top_cities)))

In [4]:
for state_index, state_row in states.iterrows():
    for city_index, city_row in top_cities.iterrows():
        if city_row['Code'] == state_row['Code']:
            state_row['Cities'].append(city_row['City'])

In [7]:
def collect_affiliation_data(city,state):
    url = 'https://www.bestplaces.net/voting/city/'+state+'/'+city
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    container = html_soup.find_all('div', class_ = 'col-md-12')
    output = container[1].find('b').text
    bad_chars = ['.', ','] 
    for i in bad_chars : 
        output = output.replace(i, '') 
    try: 
        return(output.split(' is ')[1].lower())
    except:
        return('NA')

In [27]:
outputlist = []
citylist = []
statelist = []

In [28]:



for state_index, state_row in states2.iterrows():
    state = state_row['State']
    for city in state_row['Cities']:
        cityname = city.replace(" ","_")
        try:
            outputlist.append(collect_affiliation_data(cityname,state))
        except:
            outputlist.append('NA')
        citylist.append(city)
        statelist.append(state_row['State'])
        print(state)
        print(city)


New York
New York 
New York
Buffalo 
New York
Rochester 
New York
Yonkers 
New York
Syracuse 
New York
Albany 
New York
New Rochelle 
New York
Mount Vernon 
New York
Schenectady 
New York
Utica 
New York
White Plains 
New York
Hempstead
New York
Niagara Falls 
New York
Troy 
New York
Binghamton 
New York
Freeport
New York
Valley Stream
New York
Long Beach 
New York
Rome 
New York
North Tonawanda 
New York
Ithaca 
New York
Poughkeepsie 
New York
Jamestown 
New York
Elmira 
New York
Saratoga Springs 
New York
Lindenhurst
New York
Port Chester
New York
Newburgh 
New York
Watertown 
New York
Auburn 
New York
Glen Cove 
New York
Harrison
New York
Spring Valley
New York
Middletown 
New York
Peekskill 
New York
Rockville Centre
New York
Ossining
New York
Kingston 
New York
Garden City
New York
Kiryas Joel
New York
Lockport 
New York
Lynbrook
New York
Plattsburgh 
New York
Mineola
New York
Cortland 
New York
Mamaroneck
New York
Scarsdale
New York
Lackawanna 
New York
Amsterdam 
New York
Massap

In [29]:
len(outputlist)

186

In [30]:
outputdf = pd.DataFrame(outputlist)
citydf = pd.DataFrame(citylist)
statedf = pd.DataFrame(statelist)

In [31]:
df_out = pd.concat([outputdf,citydf,statedf],axis=1)

In [33]:
df_out.head()

,0,0,0
0,moderately liberal,New York,New York
1,moderately liberal,Buffalo,New York
2,moderately liberal,Rochester,New York
3,moderately liberal,Yonkers,New York
4,somewhat liberal,Syracuse,New York


In [34]:

df_out.to_csv('output.csv')